In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import skfuzzy as fuzz

# Load the image
image_path = "9.jpg"
image = Image.open(image_path)
track_value = [165,2,25]

# Read coordinates from the CSV file
for ii in track_value :
    df = pd.read_csv("Trkwise_Framewise.csv")
    df_track = df[df['Track'] == ii]  # Filter for Track == 14 and first 1000 frames

    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image)

    # Initialize lists to store angles and coordinates
    angles = []
    resultant_vectors = []
    coordinates = list(zip(df_track['xc'], df_track['yc']))

    # Calculate angles between points using a window size of 1
    for i in range(len(coordinates) - 1):
        p0 = np.array(coordinates[i])
        p1 = np.array(coordinates[i + 1])
        v0 = p1 - p0
        angles.append(np.degrees(np.arctan2(-v0[1], v0[0])))

    # Define membership functions for angles
    angle_range = np.arange(-180, 181, 1)

    angle_low = fuzz.trimf(angle_range, [-180, -90, 0])
    angle_low_medium = fuzz.trimf(angle_range, [-135, -45, 0])
    angle_medium = fuzz.trimf(angle_range, [-90, 0, 90])
    angle_medium_high = fuzz.trimf(angle_range, [0, 45, 135])
    angle_high = fuzz.trimf(angle_range, [90, 180, 180])

    # Apply fuzzy rules
    rule1 = fuzz.interp_membership(angle_range, angle_low, angles)
    rule2 = fuzz.interp_membership(angle_range, angle_low_medium, angles)
    rule3 = fuzz.interp_membership(angle_range, angle_medium, angles)
    rule4 = fuzz.interp_membership(angle_range, angle_medium_high, angles)
    rule5 = fuzz.interp_membership(angle_range, angle_high, angles)

    # Combine the rules using fuzzy OR
    or_result = np.fmax(np.fmax(np.fmax(rule1, rule2), np.fmax(rule3, rule4)), rule5)
    print(len(or_result))
    direction_range = np.arange(-180, 181, 361/len(or_result))
    print(len(direction_range))
    # Defuzzify to obtain direction
    defuzzified = fuzz.defuzz(direction_range, or_result, 'centroid')

    # Define direction based on defuzzified value
    def get_direction(defuzzified_value):
        if defuzzified_value <= -22.5 or defuzzified_value > 337.5:
            return 'E'
        elif 22.5 < defuzzified_value <= 67.5:
            return 'NE'
        elif 67.5 < defuzzified_value <= 112.5:
            return 'N'
        elif 112.5 < defuzzified_value <= 157.5:
            return 'NW'
        elif 157.5 < defuzzified_value <= 202.5:
            return 'W'
        elif 202.5 < defuzzified_value <= 247.5:
            return 'SW'
        elif 247.5 < defuzzified_value <= 292.5:
            return 'S'
        else:
            return 'SE'

    # Plot resultant vectors on the image with fuzzy logic results
    for i, (x, y) in enumerate(coordinates[:-1]):
        defuz_value = fuzz.defuzz(direction_range[i], or_result[i], 'centroid')
        direction = get_direction(defuz_value)
        # You need to compute the vector corresponding to each point if you want to plot it
        # Example:
        p0 = np.array([x, y])
        p1 = np.array([coordinates[i+1][0], coordinates[i+1][1]])
        vector = p1 - p0
        ax.quiver(x, y, vector[0], vector[1], angles='xy', scale_units='xy', scale=1, color='red', label=f"Fuzzy: {direction}")


# Show the image with resultant vectors
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Resultant Vectors with Fuzzy Logic')
# plt.legend()
plt.show()
